In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings, Pair_Hinge_Settings, Triplet_Distance_Settings
from features.models import GCNNSmall0, GCNNMedium0, GCNNMedium1, GCNNLarge0
from learn import get_feature_model, train_features, test_features
from datasets import load_cifar10

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar10(augment=True)
d=data; s=triplet_samplers

CIFAR-10 loaded in 103 seconds


In [3]:
tf.reset_default_graph()
f, f_params = get_feature_model(feature_model=GCNNLarge0, settings=Triplet_Distance_Settings,
                                num_features=10, data_format=[32, 32, 3],
                                f_lr=0.03, f_reg=0.001, f_scope="d_gl0_cifar10_10")
fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "d_gl0_cifar10_10"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fs.restore(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    train_features(sess=sess, feature_model=f, samplers=s, num_steps=25000, keep_prob=0.6, f_params=f_params)
    fs.save(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    test_features(sess=sess, feature_model=f, samplers=s, f_params=f_params)

INFO:tensorflow:Restoring parameters from ./models/cifar10/d_gl0_cifar10_10.ckpt
begin training features
	step 0: train loss 0.210856, train error 0.0666667, valid loss 0.15528, valid error 0.0433334
	step 200: train loss 0.188704, train error 0.0533333, valid loss 0.193844, valid error 0.0566667
	step 400: train loss 0.188393, train error 0.0433334, valid loss 0.191769, valid error 0.05
	step 600: train loss 0.203301, train error 0.0466667, valid loss 0.187317, valid error 0.0433334
	step 800: train loss 0.240867, train error 0.06, valid loss 0.209224, valid error 0.0533333
	step 1000: train loss 0.160787, train error 0.0266666, valid loss 0.186695, valid error 0.0466667
	step 1200: train loss 0.200226, train error 0.0566667, valid loss 0.169639, valid error 0.0333334
	step 1400: train loss 0.186714, train error 0.05, valid loss 0.186261, valid error 0.0466667
	step 1600: train loss 0.190554, train error 0.0433334, valid loss 0.164511, valid error 0.0333334
	step 1800: train loss 0.21

In [4]:
from classes.models import LinearClassifier0, LinearClassifier1, LinearClassifier2, LinearClassifier3, TwoLayerClassifier0
from learn import get_feature_model, get_classifier, train_features, train_classifier, test_model, test_features

In [5]:
tf.reset_default_graph()
f, f_params = get_feature_model(feature_model=GCNNLarge0, settings=Triplet_Distance_Settings,
                                num_features=10, data_format=[32, 32, 3],
                                f_lr=0.07, f_reg=0.001, f_scope="d_gl0_cifar10_10")
c, c_params = get_classifier(classifier=LinearClassifier0, num_features=10, num_classes=10, 
                             c_lr=0.01, c_reg=0.001, c_scope="meh", f_params=f_params)
fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "d_gl0_cifar10_10"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fs.restore(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    train_classifier(sess=sess, feature_model=f, classifier=c, data=d, samplers=s, num_steps=10000, 
                     f_params=f_params, c_params=c_params)
    test_model(sess=sess, feature_model=f, classifier=c, samplers=s, data=d, f_params=f_params, c_params=c_params)

INFO:tensorflow:Restoring parameters from ./models/cifar10/d_gl0_cifar10_10.ckpt
begin training classifier
	step 0: train loss 2.41882, train error 0.997619
	step 100: train loss 2.24127, train error 0.99
	step 200: train loss 2.06879, train error 0.800476
	step 300: train loss 1.92497, train error 0.619048
	step 400: train loss 1.798, train error 0.448571
	step 500: train loss 1.70681, train error 0.411429
	step 600: train loss 1.59792, train error 0.33
	step 700: train loss 1.51048, train error 0.254286
	step 800: train loss 1.44057, train error 0.25
	step 900: train loss 1.37654, train error 0.23619
	step 1000: train loss 1.29041, train error 0.210952
	step 1100: train loss 1.24973, train error 0.218571
	step 1200: train loss 1.18802, train error 0.20619
	step 1300: train loss 1.15546, train error 0.195714
	step 1400: train loss 1.1076, train error 0.210476
	step 1500: train loss 1.08922, train error 0.206667
	step 1600: train loss 1.01999, train error 0.184762
	step 1700: train los